In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
from os import listdir
from os.path import isfile, join
import logging
import re
import pandas as pd
import glob
import sqlite3
import time
import datetime

pd.options.mode.chained_assignment = None

print("Aftrap functie defs")

def rawtxtjesse_to_df(directory):
    
    "Create list with all .txt files only in given directory"
    txt_list = []
    for filename in glob.glob(directory + '/*.txt'):
        txt_list.append(filename)
     
    "Create dataframe to be returned"
    df_main = pd.DataFrame(columns=['datetime', 'user', 'message'])
    
    'Loop through list with .txt files'
    for filename in txt_list:
        file = open(filename, encoding='utf-8')
        read_lines = file.readlines()
        list_of_lists = [[line] for line in read_lines]
        df1 = pd.DataFrame.from_records(list_of_lists)
        df1.columns = ['raw_line']

        "Make dates_only series"
        sep1 = ' - '
        dates_only = df1.raw_line.str.split(sep1).str[0]
        dates_only = dates_only.rename('datetime')

        "Pass rest"
        raw_minus_date = df1.raw_line.str.split(sep1).str[1:].str.join('')

        "Make names_only series"
        sep2 = ': '
        names_only = raw_minus_date.str.split(sep2).str[0]
        names_only = names_only.rename('user')

        "Pass rest"
        rest = raw_minus_date.str.split(sep2)                                     .str[1:]                                     .str                                     .join(sep2)

        "Make messages_only series"
        messages_only = rest.str.strip('\n')
        messages_only = messages_only.rename('message')

        "Make seperated dataframe from series"
        df_2 = pd.concat([ dates_only , names_only , messages_only ], axis=1 )

        "Filter seperated dataframe where message is empty"
        df_3 = df_2[df_2['message'] != '']

        "Try except to format datetime"
        try:
            df_3.loc[:,'datetime'] = pd.to_datetime(df_3['datetime'], format='%d-%m-%y %H:%M')
            '#print(f"Eerste format succesvol voor {filename}")'
        except ValueError:
            '#print(f"Eerste format gefaald voor {filename}")' 
            try:
                df_3.loc[:,'datetime'] = pd.to_datetime(df_3['datetime'], format='%d-%m-%Y %H:%M')
                '#print(f"Tweede format succesvol voor {filename}")'
            except ValueError:
                'print(f"Tweede format gefaald voor {filename}")'
        
        "Concat df to df_main"
        df_main = pd.concat([df_main, df_3])
        
    "Drop duplicates"
    df_main = df_main.drop_duplicates() 
    
    "Adjust user column names"
    df_main.loc[df_main.user == 'Jesse', 'user'] = 'Jesse Niëns'
    df_main.loc[df_main.user == 'Adam Van Der Maat', 'user'] = 'Adam van der Maat'
    
    return df_main


def rawtxtadam_to_df(directory):
    txt_list = []
    for filename in glob.glob(directory + '/*.txt'):
        txt_list.append(filename)
    
    df_main = pd.DataFrame(columns=['datetime', 'user', 'message'])
    
    for filename in txt_list:
        file = open(filename, encoding='utf-8')

        read_lines = file.readlines()  # This makes it a list with an item per record
        list_of_lists = [[line] for line in read_lines]
        df = pd.DataFrame.from_records(list_of_lists)
        df.columns = ['raw_line']
        
        sep1 = ':\d\d] '
        dates_only = (
            # splits ay first regexobject (seconds), returns series with lists    
            df.raw_line.str.split(sep1) \

            # takes first item of list per row, returns series
            .str[0] \

            # skips first index of string
            .str[1:] \

            # strips the left '[', returns series with date
            .str.lstrip('[')
        )
        
        sep2 = '] '
        
        raw_minus_date = df.raw_line.str.split(sep2).str[1:].str.join('')
        
        sep3 = ': '
        names_only = raw_minus_date.str.split(sep3).str[0]
        
        rest = raw_minus_date.str.split(sep3)                         .str[1:]                         .str                         .join(sep3)
        
        df['date'] = dates_only
        df['user'] = names_only
        df['message'] = rest
        df_1 = df.drop(columns=['raw_line'])
        df2 = df_1[df_1['message'] != '']
        df2.loc[:,'message'] = df2.loc[:,'message'].str.strip('\n')
        df2['datetime'] = pd.to_datetime(df2['date'], format='%d-%m-%y %H:%M')
        df2 = df2.drop(columns=['date'])
        df3 = df2[['datetime', 'user', 'message']]

        df_main = pd.concat([df_main, df3])
    
    df_main.loc[df_main.user == 'Maurits', 'user'] = 'Maurits Edens'
    df_main.loc[df_main.user == 'Adam Van Der Maat', 'user'] = 'Adam van der Maat'
    df_main.loc[df_main.user == 'Hendrik Strabbinb', 'user'] = 'Hendrik Strabbing'
    df_main.loc[df_main.user == 'Ramses', 'user'] = 'Ramses Walon'
    df_main.loc[df_main.user == 'Jesse', 'user'] = 'Jesse Niëns'
    df_main.loc[df_main.user == 'Nils De Koning', 'user'] = 'Nils Koning'
    df_main.loc[df_main.user == 'Django', 'user'] = 'Django Walon'
    df_main.loc[df_main.user == '\u202a+31\xa06\xa081031891\u202c', 'user'] = 'Lenny von Liechtenstein'
    df_main.loc[df_main.user == 'Koen De Boer', 'user'] = 'Koen de Boer'
    df_main = df_main.drop_duplicates()   
    
    return df_main


def combine_dfs(df1, df2):
    
    df3 = pd.concat([df1, df2])
    df4 = df3.drop_duplicates() 
    df5 = df4[df4['user'] != 'Fun Fun Fun']
    df6 = df5.sort_values('datetime')
    df6.loc[:,'datetime'] = df6.datetime.dt.strftime('%Y-%m-%d %H:%M')
    
    return df6


def convert_df_to_tuplearray(df):
    
    tuples = [tuple(x) for x in df.values]
    
    return tuples


def create_table():
    conn = sqlite3.connect('/database/fun.db')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS fun_db(datestamp TEXT, user TEXT, message TEXT)')
    "print('Table created')"

def drop_table():
    conn = sqlite3.connect('/database/fun.db')
    c = conn.cursor()
    c.execute('DROP TABLE IF EXISTS fun_db')
    "print('Table dropped')"

    
def data_executemany(record_list):        
    conn = sqlite3.connect('/database/fun.db')
    c = conn.cursor()
    sqlite_insert_query = """INSERT INTO fun_db
                      (datestamp, user, message) 
                      VALUES (?, ?, ?);"""

    c.executemany(sqlite_insert_query, record_list)
    conn.commit()
    c.close()
    conn.close()

print("Nu begint while loop")
x = 0

while True:
    print("While loop begonnen")
    x = x + 1
    drop_table()
    create_table()
    df_jesse = rawtxtjesse_to_df('/data_files/jesse')
    df_adam = rawtxtadam_to_df('/data_files/adam')
    df_main = combine_dfs(df_jesse, df_adam)
    tuples = convert_df_to_tuplearray(df_main)   
    data_executemany(tuples)
    
    print("executemany uitgevoerd")
    print(f'Number of records: {len(df_main)}')
    print(f'Times refresht: {x}')
    print("Lekker bezig")
    time.sleep(60)
    